In [4]:
import pandas as pd

# Домашнее задание

Отлично! Теперь поработаем с данными о поездках на такси в Нью-Йорке, где также имеется информация о погодных условиях и выходных днях.  <br>
•	pickup_dt – период с точностью до часа <br>
•	pickup_month – месяц <br>
•	borough – район Нью-Йорка, из которого был сделан заказ (5 районов + аэропорт) <br>
•	pickups – число поездок за период (час) <br>
•	hday – является ли день праздничным/выходным; Y - да,  N - нет <br>
•	spd – скорость ветра в милях в час <br>
•	vsb – видимость <br>
•	temp – температура в градусах Фаренгейта <br>
•	dewp – точка росы по Фаренгейту <br>
•	slp – давление <br>
•	pcp_01 – количество осадков за час <br>
•	pcp_06 – количество осадков за 6 часов <br>
•	pcp_24 – количество осадков за 24 часа <br> 
•	sd – глубина снега в дюймах <br>

In [37]:
# Проверьте, сколько всего строк и столбцов в датасете
df = pd.read_csv('data/2_taxi_nyc.csv', 
                 encoding = 'windows-1251',
                sep = ',')
df.head(10)

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
5,2015-01-01 01:00:00,Jan,Staten Island,6,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
6,2015-01-01 01:00:00,Jan,NaN,4,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
7,2015-01-01 02:00:00,Jan,Bronx,120,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
8,2015-01-01 02:00:00,Jan,Brooklyn,1229,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
9,2015-01-01 02:00:00,Jan,EWR,0,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0


In [12]:
# Выберите тип колонок, преобладающий в датасете
df.dtypes.value_counts()

float64    9
object     4
int64      1
Name: count, dtype: int64

In [14]:
# В названиях столбцов pcp 01, pcp 06, pcp 24 
# встречается пробел. Это не очень удобно, 
# если в дальнейшем вы планируете обращаться к колонкам через точку
# Замените пробел в названиях на знак нижнего подчеркивания. 
df = df.rename(columns={'pcp 01' : 'pcp_01',
                   'pcp 06' : 'pcp_06',
                   'pcp 24' : 'pcp_24'})
df.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp_01', 'pcp_06', 'pcp_24', 'sd'],
      dtype='object')

In [18]:
# Посмотреть сколько раз встречается каждый из районов
df.borough.value_counts()

borough
Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: count, dtype: int64

In [16]:
#Выяснить, из какого района совершено наибольшее количество поездок
df.pickups.sum()

14265773

In [22]:
# Больше всего поездок из Манхэтэна
pickups_borough = df.groupby('borough', as_index = False) \
                .agg({'pickups' : 'sum'}) \
                .sort_values('pickups', ascending = False)
pickups_borough

,borough,pickups
3,Manhattan,10367841
1,Brooklyn,2321035
4,Queens,1343528
0,Bronx,220047
5,Staten Island,6957
2,EWR,105


In [24]:
# Метод at [индекс строки, название столбца]
# Можно вытащить значение столбца, зная индекс строки
min_pickups = pickups_borough.at[pickups_borough.pickups.idxmin(), 'borough']
min_pickups

'EWR'

In [26]:
# Продолжим изучение данных и посмотрим на число поездок в выходные дни. 
# Сгруппируйте данные по двум признакам: району города и является ли день выходным 
# (колонки borough и hday). Сравните среднее число поездок, и выберите районы, 
# из которых по праздникам в среднем поступает больше заказов, чем в обычные дни. 
pickups_hday = df.groupby(['borough','hday']) \
            .agg({'pickups' : 'mean'})
pickups_hday

pickups
borough       hday             
Bronx         N       50.771073
              Y       48.065868
Brooklyn      N      534.727969
              Y      527.011976
EWR           N        0.023467
              Y        0.041916
Manhattan     N     2401.302921
              Y     2035.928144
Queens        N      308.899904
              Y      320.730539
Staten Island N        1.606082
              Y        1.497006

In [30]:
# Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные
# значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor.
pickups_mon = df.groupby(['borough','pickup_month']) \
            .agg({'pickups' : 'sum'}) \
            .sort_values('pickups', ascending = False)
pickups_mon

pickups
borough       pickup_month         
Manhattan     Jun           1995388
              May           1888800
              Feb           1718571
              Mar           1661261
              Apr           1648278
              Jan           1455543
Brooklyn      Jun            482466
              May            476087
              Apr            378095
              Mar            346726
              Feb            328650
              Jan            309011
Queens        Jun            286311
              May            275893
              Mar            219561
              Apr            216857
              Feb            185695
              Jan            159211
Bronx         May             53037
              Jun             49006
              Apr             34617
              Mar             32232
              Feb             28694
              Jan             22461
Staten Island Jun              1673
              May              1517
              Apr              1068
              Mar               975
              Feb               903
              Jan               821
EWR           Jun                29
              May                27
              Mar                14
              Feb                14
              Jan                11
              Apr                10

In [32]:
# Попрактикуемся в написании собственных функций и их применении к датафрейму.
# Поскольку данные о поездках в Нью-Йорке, температура представлена в градусах Фаренгейта. 
#Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F, 
# и возвращает значения, переведенные в градусы Цельсия.
def temp_to_celcius(series):
    return round((series - 32) / 1.8,2)
df['temp_C'] = temp_to_celcius(df.temp)
df['temp_C']

0        -1.11
1        -1.11
2        -1.11
3        -1.11
4        -1.11
         ...  
29096    23.89
29097    23.89
29098    23.89
29099    23.89
29100    23.89
Name: temp_C, Length: 29101, dtype: float64

# Минипроект

### Задачи <br>
1.	Импортируйте библиотеку pandas как pd. Загрузите датасет bookings.csv с разделителем ;. Проверьте размер таблицы, типы переменных, а затем выведите первые 7 строк, чтобы посмотреть на данные. <br><br>
2.	Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания. <br><br>
3.	Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5. <br><br>
4.	На сколько ночей в среднем бронируют отели разных типов? <br><br>
5.	Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете? <br><br>
6.	Проанализируйте даты запланированного прибытия. – На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?– Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов <br><br>
7.	Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение? <br><br>
8.	Создайте колонку total_kids, объединив children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим? <br><br>
9.	Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.

### Описание данных
Имеются следующие переменные: <br>
•	Hotel – тип отеля (City Hotel или Resort Hotel)   <br>
•	Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным<br>
•	Lead time – количество дней, прошедших между датой бронирования и датой прибытия  <br>
•	Arrival full date – полная дата прибытия<br>
•	Arrival date year – год прибытия  <br>
•	Arrival date month – месяц прибытия  <br>
•	Arrival date week number – номер недели прибытия<br>
•	Arrival date day of month – день прибытия<br>
•	Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле<br>
•	Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле<br>
•	Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)<br>
•	Adults – число взрослых<br>
•	Children – число детей<br>
•	Babies – число младенцев <br>
•	Meal – выбранный тип питания<br>
•	Country – страна происхождения клиента<br>
•	Reserved room type – тип зарезервированного номера<br>
•	Assigned room type – тип полученного номера (может отличаться от забронированного)<br>
•	Customer type – тип бронирования<br>
•	Reservation status – значение последнего статуса брони: Canceled - было отменено клиентом; Check-Out - клиент зарегистрировался, но уже покинул отель; No-Show - клиент не зарегистрировался и сообщил администрации отеля причину<br>
•	Reservation status date – дата обновления статуса<br>

In [41]:
hotels = pd.read_csv("data/2_bookings.csv", sep = ';')
hotels.head()

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [43]:
hotels.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [45]:
hotels.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [47]:
#2.	Приведите названия колонок к нижнему регистру и 
# замените пробелы на знак нижнего подчеркивания.
hotels.columns
def rename_columns(column):
    new_col = []
    for name in column:
        new_col.append(name.lower().replace(' ', '_'))
    return new_col
hotels.columns = rename_columns(hotels.columns)
hotels.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

In [49]:
hotels.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [51]:
# 3.	Пользователи из каких стран совершили наибольшее число успешных бронирований?
# Укажите топ-5.
# query можно заменить на bookings.loc[bookings.is_canceled == 0]
bookings_countries = hotels.query("is_canceled == 0").country.value_counts()[0:5]
bookings_countries

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: count, dtype: int64

In [53]:
# 4.  На сколько ночей (stays_total_nights)  в среднем бронируют отели типа City Hotel? Resort Hotel? 
# Запишите полученные значения в пропуски с точностью до 2 знаков после точки.
avg_total_nights = hotels.groupby('hotel').agg({'stays_total_nights' : 'mean'})
avg_total_nights

,stays_total_nights
hotel,
City Hotel,2.978142
Resort Hotel,4.318547


In [55]:
# 5.  На сколько ночей (stays_total_nights)  в среднем бронируют отели типа City Hotel? Resort Hotel? 
# Запишите полученные значения в пропуски с точностью до 2 знаков после точки.
avg_total_nights = hotels.groupby('hotel').agg({'stays_total_nights' : 'mean'})
avg_total_nights

,stays_total_nights
hotel,
City Hotel,2.978142
Resort Hotel,4.318547


In [57]:
len(hotels.loc[hotels.assigned_room_type != hotels.reserved_room_type])

14917

In [59]:
# 6. Теперь проанализируйте даты запланированного прибытия (arrival_date_year).
# На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017?
count_per_month = hotels.groupby(['arrival_date_year','arrival_date_month']) \
                .agg({'hotel' : 'count'}).rename(columns = {'hotel' : 'count'}) \
                .sort_values(['arrival_date_year','count'], ascending = False)
count_per_month

count
arrival_date_year arrival_date_month       
2017              May                  6313
                  April                5661
                  June                 5647
                  July                 5313
                  March                4970
                  August               4925
                  February             4177
                  January              3681
2016              October              6203
                  May                  5478
                  April                5428
                  September            5394
                  June                 5292
                  August               5063
                  March                4824
                  July                 4572
                  November             4454
                  February             3891
                  December             3860
                  January              2248
2015              September            5114
                  October              4957
                  August               3889
                  December             2920
                  July                 2776
                  November             2340

In [61]:
# 7. Сгруппируйте данные по годам, а затем проверьте, на какой месяц (arrival_date_month) 
# бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 
hotels.query('hotel == "City Hotel" and is_canceled == 1') \
      .groupby('arrival_date_year')['arrival_date_month'] \
      .value_counts()

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

In [63]:
# 8. Посмотрите на числовые характеристики трёх колонок: adults, children и babies. 
# Какая из них имеет наибольшее среднее значение?
print("Взрослые :", hotels.adults.mean())
print("Дети :", hotels.children.mean())
print("Малыши :", hotels.babies.mean())

Взрослые : 1.8564033838679956
Дети : 0.10388990333874994
Малыши : 0.007948739425412514


In [65]:
# 9. Создайте колонку total_kids, объединив столбцы children и babies. 
# Для отелей какого типа среднее значение переменной оказалось наибольшим?
hotels['total_kids'] = hotels['children'] + hotels['babies']
avg_kids = hotels.groupby('hotel').total_kids.mean()
avg_kids

hotel
City Hotel      0.096311
Resort Hotel    0.142586
Name: total_kids, dtype: float64

In [69]:
# 10. Не все бронирования завершились успешно (is_canceled), поэтому можно посчитать,
# сколько клиентов было потеряно в процессе. 
#Иными словами, посчитать метрику под названием Churn Rate.

# Churn rate (отток, коэффициент оттока)
# Иными словами, представляет собой отношение количества ушедших пользователей 
# к общему количеству пользователей, выраженное в процентах.

# В нашем случае Churn Rate - это процент клиентов, которые отменили бронирование. 
# Давайте посмотрим, как эта метрика связана с наличием детей у клиентов!
# Создайте колонку has_kids, которая принимает значение True, 
# если клиент при бронировании указал хотя бы одного ребенка (total_kids), 
# в противном случае – False. 
# Далее проверьте, среди какой группы пользователей показатель оттока выше. 
# В качестве ответа укажите наибольший % оттока, 
# округленный до 2 знаков после точки (то есть доля 0.24563 будет 24.56% и в ответ пойдёт 24.56)
hotels['has_kids'] = hotels['total_kids'] > 0